In [1]:
from pianoq.misc.mplc_lab_imports import * 
%load_ext autoreload
%autoreload 2

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
can't use pco camera


# Camera things

In [4]:
cam = PCOCamera()
cam.set_exposure_time(0.5)

ml = MPLCAligner(use_cam=False)
ml.update('1to5w4f', '5to11w8', pi_steps_x=[0], pi_steps_plane=1)

In [47]:
roi=(750, 350, 1200, 850)
live_cam(cam, roi=roi)

In [42]:
roi=(750, 350, 1200, 850)
cam.show_image(roi=roi)

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [48]:
roi=(750, 350, 1200, 850)
name = 'U1_best.fits'
cam.save_image(path=rf'{dir_path}\{name}', roi=roi)
cam.show_image(roi=roi)

(<Figure size 640x480 with 1 Axes>, <Axes: >)

In [ ]:
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
path = rf'{dir_path}\{timestamp}_{name}.cam'
cam.save_image(path)

In [ ]:
cam.close()

# MPLC 

In [2]:
modes_to_keep = np.array([3, 8, 13, 18, 23, 28, 33, 38, 43, 48])

mplc = MPLCDevice()
mplc.restore_location()
print('Got MPLC')
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_10_14_00_34_old.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_25_13_19_15_extra_1cm.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_17_first_diffuser\2024_09_17_12_19_58_diffuser_d1d.masks"
# masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_29_14_27_10_U1U.masks"  # with dead zone 
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 

msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
# phases = np.random.uniform(0, 2*np.pi, size=50)
# masks = add_phase_input_spots(masks, phases)
mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=35)
mplc.restore_location()

Got MPLC


# motors - actually now use in separate notebook

In [3]:
backlash = 0
wait_after_move = 0.3

zaber_ms = ZaberMotors(backlash=backlash, wait_after_move=wait_after_move)
mxs = zaber_ms.motors[1]
mys = zaber_ms.motors[0]
print('Got Zaber motors')

mxi = ThorlabsKcubeDC(thorlabs_x_serial, backlash=backlash, wait_after_move=wait_after_move)
myi = ThorlabsKcubeStepper(thorlabs_y_serial, backlash=backlash, wait_after_move=wait_after_move)
print('Got Thorlabs motors')

Got Zaber motors
Got Thorlabs motors


In [131]:
mxi.move_absolute(9.30)
myi.move_absolute(4.05)
print(mxi.get_position())
print(myi.get_position())

9.3001
4.05


In [142]:
mxs.move_absolute(13.03)
mys.move_absolute(6.45)
print(mxs.get_position())
print(mys.get_position())

13.0300095
6.449996625


In [147]:
myi.move_relative(0.01)

In [4]:
print(mxi.get_position()) # 9.2701
print(myi.get_position()) # 3.3100

9.3
4.05


BS
- best idler: ~9K
- best signal: ~8.3K 

Mirror:
- best idler: 
- best signal: 


## Scans to see what happened 

In [4]:
# Timetagger
tt = QPTimeTagger(integration_time=1, remote=True,
                  single_channel_delays=TIMETAGGER_DELAYS, coin_window=TIMETAGGER_COIN_WINDOW)

### idler scan

In [8]:
integration_time = 1
start_x = 8.9
end_x = 9.6
start_y = 2.7
end_y = 5.2

mxi.wait_after_move = 0
myi.wait_after_move = 0
# start_x = 8.5
# end_x = 9.5
# start_y = 2
# end_y = 4

resolution = 1 
x_pixels = 14 // resolution
y_pixels = 30 // resolution
pixel_size_x = 0.05 * resolution
pixel_size_y = 0.05 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_idler_path = fr'{dir_path}\{timestamp}_idler_6_spots_again.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_idler_path)   
_ = run_in_thread(scanner.scan, x_motor=mxi, y_motor=myi, ph=tt)

### signal scan

In [9]:
integration_time = 1
start_x = 12.65
end_x = 13.35
start_y = 5.5
end_y = 8

mxs.wait_after_move = 0.1
mys.wait_after_move = 0.1

resolution = 1 
x_pixels = 14 // resolution
y_pixels = 50 // resolution
pixel_size_x = 0.05 * resolution
pixel_size_y = 0.05 * resolution

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
singles_signal_path = fr'{dir_path}\{timestamp}_signal_initial_spots.scan'

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=2e-9, saveto_path=singles_signal_path)   
_ = run_in_thread(scanner.scan, x_motor=mxs, y_motor=mys, ph=tt)

In [98]:
mimshow(scanner.single1s, vmax=3700)
mimshow(scanner.single2s, vmax=1000)

(<Figure size 640x480 with 2 Axes>, <Axes: >)

# New WFM 

## with defocus distance and dead zone 

In [6]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1 
dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror'
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path =    fr'{dir_path}\{timestamp}_U1U.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_U1U.mplcsim'
mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, dead_middle_zone=450e-6, last_plane_extra_dist=8.4e-3)
mplc_sim.res.saveto(mplc_res_path, smaller=True)

{'wavelength': 8.1e-07, 'dist_after_plane': array([0.087 , 0.087 , 0.087 , 0.087 , 0.138 , 0.087 , 0.087 , 0.087 ,
       0.087 , 0.0954]), 'N_iterations': 15, 'Nx': 140, 'Ny': 360, 'dx': 1.25e-05, 'dy': 1.25e-05, 'max_k_constraint': 0.15, 'min_log_level': 2, 'size_factor': 3, 'use_mask_offset': True, 'symmetric_masks': False, 'active_planes': array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True]), 'N_modes': 10}
1


100%|██████████| 15/15 [02:45<00:00, 11.01s/it]


In [7]:
mplc_sim.res.show_overlap()

In [8]:
mplc_sim.res._calc_fidelity()
mplc_sim.res.fidelity

0.9896838134848069

In [9]:
mplc_sim.res.show_all()

# mark spot locations 

In [47]:
from pianoq_results.scan_result import ScanResult
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
locs_signal_path = fr'{dir_path}\{timestamp}_s2_sig_mirror.locs'
locs_idler_path = fr'{dir_path}\{timestamp}_s1_idl_mirror.locs'

res_idl = ScanResult(fr'{dir_path}\2024_09_30_09_26_08_idler_5_spots_tilt_35_deadzone_450_fine.scan')
res_sig = ScanResult(fr'{dir_path}\2024_09_30_09_28_15_signal_5_spots_tilt_35_deadzone_450_fine.scan')
# res_idl.show_singles()
# res_sig.show_singles()

res_sig.get_xys(2, saveto_path=locs_signal_path)
res_idl.get_xys(1, saveto_path=locs_idler_path)

Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_12_30_05_s2_sig_mirror.locs
Saved to G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_12_30_05_s1_idl_mirror.locs


[(9.29882867132867, 3.948489010989011),
 (9.332395104895104, 4.296141358641359),
 (9.370756743256742, 4.6294080919080915),
 (9.37794955044955, 4.962674825174825),
 (9.416311188811187, 5.303134365634366)]

# Phases

In [ ]:
# masks 
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=35)

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
phases_path = fr'{dir_path}\{timestamp}_new_mirror.phases'
integration_time = 30

phases_result = PhaseFinderResult(fr"{dir_path}\2024_09_30_12_53_59_new_mirror.phases")


pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=phases_path,
                 modes_to_keep=modes_to_keep, intial_phases=phases_result.phases, coin_window=1e-9, no_hw_mode=True, linear_tilts=True, plane_10_tilts=35)

pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)

# locs 
locs_sig = np.load(fr'{dir_path}\2024_09_30_12_30_05_s2_sig_mirror.locs')['locs']
locs_idl = np.load(fr'{dir_path}\2024_09_30_12_30_05_s1_idl_mirror.locs')['locs']

# i,j of corr matrix that is supposed to be strong 
i = 1
j = 1
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])        
mplc.restore_location()
time.sleep(1)
pf.find_phases()

# Correlations

In [19]:
from pianoq.lab.mplc.phase_finder_result import PhaseFinderResult
from pianoq.lab.mplc.discrete_photon_scanner import DiscretePhotonScanner
from pianoq.simulations.mplc_sim.mplc_sim_result import MPLCMasks
from pianoq.lab.mplc.mask_utils import remove_input_modes, add_phase_input_spots
from pianoq.lab.mplc.mplc_device import MPLCDevice
import glob
import time 

backlash = 0.0
wait_after_move = 0.3
integration_time = 4
coin_window = 1.2e-9
timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

scan_path = fr'{dir_path}\{timestamp}.dscan'

locs_sig = np.load(fr'{dir_path}\2024_09_30_12_30_05_s2_sig_mirror.locs')['locs']
locs_idl = np.load(fr'{dir_path}\2024_09_30_12_30_05_s1_idl_mirror.locs')['locs']

dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path,
                            backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window, no_hw_mode=True)
dps.zaber_ms = zaber_ms
dps.m_sig_x = mxs
dps.m_sig_y = mys
dps.m_idl_x = mxi
dps.m_idl_y = myi
dps.time_tagger = tt
dps.time_tagger.set_integration_time(integration_time)
time.sleep(1)

phases_result = PhaseFinderResult(fr"{dir_path}\2024_09_30_15_06_11_new_mirror.phases")

# masks 
masks_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_25_BS_to_mirror\2024_09_30_08_56_10_U1U.masks"  # with dead zone 
msks = MPLCMasks()
msks.loadfrom(masks_path)
masks = msks.real_masks
masks = remove_input_modes(masks, modes_to_keep=modes_to_keep)
mplc.restore_location()
masks = add_phase_input_spots(masks, phases_result.phases)
mplc.load_masks(masks, linear_tilts=True, plane_10_tilts=35)

mplc.restore_location()

In [20]:
dps.scan()

beginning scan
0, 0: 14461.75, 13721.50, 8.00
0, 1: 14556.25, 9873.25, 0.75
0, 2: 14473.50, 13272.00, 2.25
0, 3: 14578.75, 12117.75, 4.00
0, 4: 14426.50, 12861.00, 1.75
1, 0: 14166.50, 13728.00, 0.50
1, 1: 14257.25, 9835.50, 2.50
1, 2: 14332.25, 13394.25, 4.50
1, 3: 14186.00, 12085.00, 1.25
1, 4: 14232.75, 12872.00, 2.00
2, 0: 15536.00, 13768.75, 2.50
2, 1: 15423.00, 9919.75, 1.25
2, 2: 15268.75, 13321.75, 4.75
2, 3: 15355.00, 12075.50, 1.75
2, 4: 15252.75, 12858.75, 2.50
3, 0: 14404.75, 13904.50, 2.75
3, 1: 14351.00, 10035.25, 1.75
3, 2: 14453.50, 13517.25, 1.25
3, 3: 14423.75, 12188.00, 4.25
3, 4: 14400.50, 12898.75, 1.75
4, 0: 15626.00, 13818.00, 0.75
4, 1: 15745.25, 9845.50, 1.50
4, 2: 15574.75, 13352.00, 1.50
4, 3: 15585.75, 12160.25, 3.75
4, 4: 15518.00, 12919.00, 5.50


In [11]:
display(locs_idl)
display(locs_sig)

array([[9.29882867, 3.94848901],
       [9.3323951 , 4.29614136],
       [9.37075674, 4.62940809],
       [9.37794955, 4.96267483],
       [9.41631119, 5.30313437]])

array([[12.97315435,  6.57471279],
       [12.9299975 ,  6.21986763],
       [12.90122627,  5.8866009 ],
       [12.88923826,  5.55333417],
       [12.87245504,  5.22246503]])

In [21]:
dps.res.show()

In [34]:
mxi.move_absolute(9.3)
myi.move_absolute(3.90)
print(mxi.get_position())
print(myi.get_position())

9.30005
3.9


In [42]:
mxs.move_absolute(12.875)
mys.move_absolute(5.22)
print(mxs.get_position())
print(mys.get_position())

12.874990125000002
5.21998575
